In [29]:
import pandas as pd
import numpy as np
import csv
import glob
import pickle


In [2]:
import rubinml

/Users/duncan/Documents/code/physics/repos/rubin_sim/rubin_sim/utils/__init__.py:6: DeprecationWarning: rubin_sim.utils is deprecated, switch to rubin_scheduler.utils
  warnings.warn("rubin_sim.utils is deprecated, switch to rubin_scheduler.utils", DeprecationWarning)


In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
basedir = '/Volumes/thesis/rubin-sim/userdata/opsim_ml/'

In [5]:
sources = pd.read_parquet(basedir+"rand_tristar_10_000_000.parquet")

In [6]:
sources

,ra,dec,gall,galb,umag,gmag,rmag,imag,zmag,ymag,mu0
0,34.566226,23.525654,147.444559,-35.179429,28.313999,26.914000,26.219000,25.916000,25.766001,25.690001,19.000000
1,36.025192,22.812159,149.301023,-35.270543,24.799000,22.309999,21.023001,20.452999,20.155001,19.990999,10.150000
2,98.822227,-35.500170,244.073226,-18.409377,29.150000,27.313000,26.385000,25.962999,25.740000,25.622000,17.549999
3,131.407899,-36.542669,257.902052,4.026084,29.433001,26.507999,24.863001,24.051001,23.538000,23.254999,14.400000
4,122.910304,-24.064698,243.452134,5.317767,30.898001,27.591999,26.065001,24.375999,23.569000,23.153999,13.000000
...,...,...,...,...,...,...,...,...,...,...,...
9993233,80.544770,-69.065555,279.682498,-33.135707,22.068001,20.915001,20.589001,20.495001,20.465000,20.445999,18.473000
9993234,78.616045,-66.435861,276.735701,-34.336724,23.424999,22.450001,22.247000,22.207001,22.212000,22.214001,18.420000
9993235,75.220742,-68.406481,279.428426,-35.164288,30.923000,28.507000,27.284000,26.740000,26.448000,26.285999,18.424000
9993236,72.833432,-65.477024,276.248054,-36.848095,24.856001,23.809000,23.344000,23.153999,23.059000,23.017000,18.356001


In [8]:
rates_file = '/Volumes/thesis/rubin-sim/userdata/opsim_ml/rates_ml_rates_1718083107_te-True_ut-5 copy.csv'
rates= {}
with open(rates_file, 'r') as f:
        print(f'opened {rates_file}')
        reader = csv.reader(f)
        next(reader) #skip header
        for line in reader:
            rates[tuple(float(_) for _ in line[:-1])]=float(line[-1])


opened /Volumes/thesis/rubin-sim/userdata/opsim_ml/rates_ml_rates_1718083107_te-True_ut-5 copy.csv


In [10]:
n_tristar = 11433322690
n_survey_events = round(sum(rates.values())*n_tristar/len(rates.values())*24*365*10)
print(n_survey_events)

336265


In [7]:
import importlib
importlib.reload(rubinml)

<module 'rubinml' from '/Users/duncan/Documents/code/physics/microlensing/rubin-sim-ml/rubinml/rubinml/__init__.py'>

In [15]:
f'{2.:0.04f}'

'2.0000'

In [27]:
masses = [0.1,1,10,100]
for mass in masses:
  events = rubinml.events.make_events(sources, 
    basedir+f'/test/ml_events_1k_{mass:.04f}sm.pickle',
    1_000, pbhmass=mass, ntoss=20000
    )

  0%|          | 0/21000 [00:00<?, ?it/s]

  0%|          | 0/21000 [00:00<?, ?it/s]

  0%|          | 0/21000 [00:00<?, ?it/s]

  0%|          | 0/21000 [00:00<?, ?it/s]

In [38]:
for eventfile in glob.glob(basedir+'test/*.pickle'):
  print(eventfile)
  with open(eventfile, 'rb') as f:
    event_df, events_info = pickle.load(f)
    full_event_df = rubinml.events.make_full_event_df(event_df, sources)
  rubinml.plots.make_event_plots(full_event_df, outdir=basedir+'test/', name=f'1k_{events_info["pbhmass"]:.04f}sm')

/Volumes/thesis/rubin-sim/userdata/opsim_ml/test/ml_events_1k_0.1000sm.pickle
/Volumes/thesis/rubin-sim/userdata/opsim_ml/test/ml_events_1k_1.0000sm.pickle
/Volumes/thesis/rubin-sim/userdata/opsim_ml/test/ml_events_1k_10.0000sm.pickle


/Users/duncan/Documents/code/physics/microlensing/rubin-sim-ml/rubinml/rubinml/plots.py:213: RuntimeWarning: Mean of empty slice.
  return events[meancol].to_numpy().mean()
/Users/duncan/mambaforge/envs/rubinsim/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/duncan/Documents/code/physics/microlensing/rubin-sim-ml/rubinml/rubinml/plots.py:213: RuntimeWarning: Mean of empty slice.
  return events[meancol].to_numpy().mean()
/Users/duncan/mambaforge/envs/rubinsim/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/duncan/Documents/code/physics/microlensing/rubin-sim-ml/rubinml/rubinml/plots.py:213: RuntimeWarning: Mean of empty slice.
  return events[meancol].to_numpy().mean()
/Users/duncan/mambaforge/envs/rubinsim/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

/Volumes/thesis/rubin-sim/userdata/opsim_ml/test/ml_events_1k_100.0000sm.pickle


In [35]:
event_df

,source_index,gall,galb,mu0,dl,umin,crossing_time,lograte
0,5908576,276.430049,-33.403614,18.440001,9.269270,4.737998,20638.493015,-24.929739
1,9291627,358.938097,-6.282468,14.350000,3.901434,3.414489,5714.813077,-25.404937
2,8554099,8.486493,5.752153,15.350000,4.643089,1.483686,10260.395680,-24.838684
3,5887341,296.182163,0.764457,15.400000,7.334447,3.303628,8677.070054,-25.650676
4,8721471,21.602849,14.985946,14.250000,3.870681,2.231915,10196.412694,-26.371465
...,...,...,...,...,...,...,...,...
995,3166481,7.541717,-11.185366,13.050000,3.955200,3.389926,1306.492280,-27.459177
996,3818345,328.289219,-5.795627,14.250000,1.028632,4.742907,13955.862797,-28.885186
997,7893676,283.641973,-33.309377,18.504999,12.859222,0.913123,16950.313575,-24.526447
998,7777079,38.468521,-4.210169,15.500000,5.448371,0.008652,10819.929000,-25.142009


In [8]:
data = np.load('test_microlensing_dm_rubinsim_1sm/baseline_v3_3_10yrs_MicrolensingMetric_detect_tE_0_30000_days_USER.npz', allow_pickle=True)

In [9]:
data['header']

array({'metric_name': 'MicrolensingMetric_detect', 'constraint': '', 'info_label': 'tE 0_30000 days', 'sim_data_name': 'baseline_v3.3_10yrs', 'date_ran': '2024-09-24', 'display_dict': {'group': None, 'subgroup': None, 'order': 0, 'caption': 'MicrolensingMetric_detect calculated on a UserPointsSlicer basis.'}, 'plot_dict': {}, 'summary_values': None, '__version__': '2.0.1.dev124+g783a21bb', '__repo_version__': None, '__fingerprint__': None, '__dependency_versions__': None},
      dtype=object)

In [13]:
list(data.keys())

['header',
 'metric_values',
 'mask',
 'fill',
 'slicer_init',
 'slicer_name',
 'slice_points',
 'slicer_n_slice',
 'slicer_shape']

In [15]:
data['metric_values'].shape

(336265,)